In [1]:
import os
import sys
import subprocess
import pandas as pd
import numpy as np
from ast import literal_eval
# os.getcwd()

In [2]:
from main import map_course_files
from main import parse_course_files
from main import build_pset_df_tex
from main import build_problem_df_tex
from main import build_pset_df_pdf

Loaded dataframes: 
worksheet_files_df:
 index name:  None 
columns:  Index(['Unnamed: 0', 'unit', 'file_count', 'filename'], dtype='object') 
149 rows:  149 

problem_sets_df:
 index name:  problem_set_ID 
columns:  Index(['filename', 'head', 'body', 'problems_list', 'problem_count',
       'problem_IDs'],
      dtype='object') 
149 rows:  149 

problems_df:
 index name:  problem_ID 
columns:  Index(['problem_set_ID', 'question'], dtype='object') 
1855 rows:  1855 

standards_df:
 index name:  None 
columns:  Index(['course', 'chapter', 'topic', 'ccss_ID'], dtype='object') 
314 rows:  314 

standards_desc_df:
 index name:  ccss_ID 
columns:  Index(['description'], dtype='object') 
131 rows:  131 



## saving and loading problem and problem set dataframes

In [28]:
problem_df.to_csv('/Users/chris/GitHub/mathai/db/problem_df.csv')
pset_df.to_csv('/Users/chris/GitHub/mathai/db/pset_df.csv')
course_file_df.to_csv('/Users/chris/GitHub/mathai/db/course_file_df.csv')

In [3]:
course_file_df = pd.read_csv('/Users/chris/GitHub/mathai/db/course_file_df.csv', index_col=0)
print('course_file_df: ', len(course_file_df), course_file_df.index.name, course_file_df.columns)

cols_into_lists = {'problem_IDs':literal_eval, 'head':literal_eval, 'body':literal_eval, 'problems_list':literal_eval, 'problem_kind':literal_eval}
pset_df = pd.read_csv('../db/pset_df.csv', index_col='problem_set_ID', converters=cols_into_lists)
print('pset_df: ', len(pset_df), pset_df.index.name, pset_df.columns)

problem_df = pd.read_csv('../db/problem_df.csv', index_col='problem_ID')
print('problem_df: ', len(problem_df), problem_df.index.name, problem_df.columns)

course_file_df:  172 None Index(['unit', 'file_count', 'filename'], dtype='object')
pset_df:  172 problem_set_ID Index(['unit', 'filename', 'problem_count', 'problem_IDs', 'problems_list',
       'problem_kind', 'head', 'body'],
      dtype='object')
problem_df:  2443 problem_ID Index(['problem_set_ID', 'question', 'kind', 'filename', 'q_len'], dtype='object')


In [3]:
# load saved dfs for test comparison
cols_into_lists = {'problem_IDs':literal_eval, 'head':literal_eval, 'body':literal_eval, 'problems_list':literal_eval, 'problem_kind':literal_eval}
pset_df2 = pd.read_csv('../db/pset_df.csv', index_col='problem_set_ID', converters=cols_into_lists)
problem_df2 = pd.read_csv('../db/problem_df.csv', index_col='problem_ID')

## Printing Problem Sets

In [4]:
print(pset_df.columns)
units = pset_df.unit.unique()
print('\nunits: ', units)

Index(['unit', 'filename', 'problem_count', 'problem_IDs', 'problems_list',
       'problem_kind', 'head', 'body'],
      dtype='object')

units:  ['01-Intro' '02-Midpoint+distance' '03-Volume+angle-bisectors'
 '04-Transversals' '05-Transformations' '06-Analytic-geometry'
 '07-Similarity' '08-Area+volume' '09-Congruence-transformations'
 '10-Trig+similarity+analytics']


In [5]:
df = pset_df.loc[22:26].copy()
df.head(1)

,unit,filename,problem_count,problem_IDs,problems_list,problem_kind,head,body
problem_set_ID,,,,,,,,
22,02-Midpoint+distance,2-5DNQ_Segments+area.tex,6,"[303, 304, 305, 306, 307, 308]",[Complete the construction of a perpendicular ...,"[question, question, newpage, question, questi...","[\begin{document}\n, \subsubsection*{2.5 Pop...",[ \item Complete the construction of a perp...


In [4]:
course_dir = '/Users/chris/GitHub/course-files/Geom2021'
df = pset_df.copy()
if 'path_plus_filename' not in df.columns:
    df['path_plus_filename'] = course_dir + '/' + df.unit + '/' + df.filename
    #print('col not found. new vals are: \n', df2['path_plus_filename'])
#else: 
    #print('found. vals are: \n', df2['path_plus_filename'])

In [5]:
df.loc[23, ['filename', 'path_plus_filename']]

filename                                            2-5HW_Perimeter.tex
path_plus_filename    /Users/chris/GitHub/course-files/Geom2021/02-M...
Name: 23, dtype: object

In [5]:
df[30:50]

,unit,filename,problem_count,problem_IDs,problems_list,problem_kind,head,body,path_plus_filename
problem_set_ID,,,,,,,,,
30,02-Midpoint+distance,2-8DN_Constructions.tex,5,"[363, 364, 365, 366, 367]",[Complete the construction of an equilateral t...,"[question, newpage, question, section, question]","[\begin{document}\n, \subsubsection*{2.8 Do ...",[\item Complete the construction of an equilat...,/Users/chris/GitHub/course-files/Geom2021/02-M...
31,02-Midpoint+distance,2-8HW_Parameter-solving.tex,6,"[368, 369, 370, 371, 372, 373]",[Find the area of $\triangle ABC$. The altitud...,"[question, question, question, newpage, questi...","[\begin{document}\n, \subsubsection*{2.8 Hom...","[\n, \n, \item Find the area of $\triangle ABC...",/Users/chris/GitHub/course-files/Geom2021/02-M...
32,02-Midpoint+distance,2-9Exam_area-distance.tex,37,"[374, 375, 376, 377, 378, 379, 380, 381, 382, ...","[Given the rectangle $ABCD$ shown below, with ...","[question, question, question, newpage, questi...","[\begin{document}\n, \subsubsection*{2.9 Exam:...","[\n, \item Given the rectangle $ABCD$ shown be...",/Users/chris/GitHub/course-files/Geom2021/02-M...
33,02-Midpoint+distance,2-9HW_Calculations.tex,8,"[411, 412, 413, 414, 415, 416, 417, 418]","[Given $\overline{ABC}$, $BC=36.9$, and $AC=87...","[question, question, question, question, newpa...","[\begin{document}\n, \subsubsection*{2.9 Homew...","[\n, \item Given $\overline{ABC}$, $BC=36.9$...",/Users/chris/GitHub/course-files/Geom2021/02-M...
34,03-Volume+angle-bisectors,3-1DN_Angle-terminology.tex,6,"[419, 420, 421, 422, 423, 424]","[As shown below, two lines intersect making fo...","[question, question, newpage, question, questi...","[\begin{document}\n, \subsubsection*{3.1 Do No...","[\n, \item As shown below, two lines interse...",/Users/chris/GitHub/course-files/Geom2021/03-V...
35,03-Volume+angle-bisectors,3-1HW_Volume+vertical-angles.tex,7,"[425, 426, 427, 428, 429, 430, 431]","[As shown below, two lines intersect making fo...","[question, question, question, newpage, questi...","[\begin{document}\n, \subsubsection*{3.1 Homew...","[\n, \item As shown below, two lines interse...",/Users/chris/GitHub/course-files/Geom2021/03-V...
36,03-Volume+angle-bisectors,3-2CW_Angle-modeling.tex,9,"[432, 433, 434, 435, 436, 437, 438, 439, 440]",[ \subsubsection*{Do Not Solve! \\\n Mod...,"[section, question, question, question, questi...","[\begin{document}\n, \subsubsection*{3.2 Cla...","[ \subsubsection*{Do Not Solve! \\\n, M...",/Users/chris/GitHub/course-files/Geom2021/03-V...
37,03-Volume+angle-bisectors,3-2DN_Area+volume.tex,7,"[441, 442, 443, 444, 445, 446, 447]",[The area of the rectangle $METS$ shown below ...,"[question, question, question, newpage, questi...","[\begin{document}\n, \subsubsection*{3.2 Do No...","[\n, \item The area of the rectangle $METS$ ...",/Users/chris/GitHub/course-files/Geom2021/03-V...
38,03-Volume+angle-bisectors,3-2HW_Segments+area.tex,6,"[448, 449, 450, 451, 452, 453]",[Complete the construction of the bisector of ...,"[question, question, newpage, question, questi...","[\begin{document}\n, \subsubsection*{3.2 Hom...","[\n, \item Complete the construction of the ...",/Users/chris/GitHub/course-files/Geom2021/03-V...


In [6]:
build_pset_df_tex(df[30:50], problem_df)

,filename,unit
pset_ID,,
30,2-8DN_Constructions.tex,02-Midpoint+distance
31,2-8HW_Parameter-solving.tex,02-Midpoint+distance
32,2-9Exam_area-distance.tex,02-Midpoint+distance
33,2-9HW_Calculations.tex,02-Midpoint+distance
34,3-1DN_Angle-terminology.tex,03-Volume+angle-bisectors
35,3-1HW_Volume+vertical-angles.tex,03-Volume+angle-bisectors
36,3-2CW_Angle-modeling.tex,03-Volume+angle-bisectors
37,3-2DN_Area+volume.tex,03-Volume+angle-bisectors
38,3-2HW_Segments+area.tex,03-Volume+angle-bisectors


In [7]:
build_pset_df_pdf(df[30:50], problem_df)

In [66]:
os.chdir('/Users/chris/GitHub/course-files/Geom2021/' + units[0])
os.getcwd()

'/Users/chris/GitHub/course-files/Geom2021/01-Intro'

In [76]:
def test_print(pset_df):
    for pset in pset_df[pset_df['unit'] == units[0]].itertuples():
        print(pset.Index)
        if 'path_plus_filename' in pset_df.columns:
            path_plus_fn = pset.path_plus_filename
        else: path_plus_fn = None
        print(path_plus_fn)
        title = (pset.filename, 'pset ID: ' + str(row.Index), 'Print to directories')
        
        #build_problem_df_tex(problem_df.loc[pset.problem_IDs], pset.filename, path_plus_fn, title)

In [86]:
for pset in df2.itertuples():
    print(pset.Index)
    if 'path_plus_filename' in df2.columns:
        path_plus_filename = pset.path_plus_filename
    else: path_plus_filename = None
    print(path_plus_filename)
    title = (pset.filename, 'pset ID: ' + str(row.Index), 'Print to directories')

22
/Users/chris/GitHub/course-files/Geom2021/02-Midpoint+distance/2-5DNQ_Segments+area.tex
23
/Users/chris/GitHub/course-files/Geom2021/02-Midpoint+distance/2-5HW_Perimeter.tex
24
/Users/chris/GitHub/course-files/Geom2021/02-Midpoint+distance/2-6DN_Distance+Perimeter.tex
25
/Users/chris/GitHub/course-files/Geom2021/02-Midpoint+distance/2-6HW_Compound-shapes.tex
26
/Users/chris/GitHub/course-files/Geom2021/02-Midpoint+distance/2-7CW_Modeling.tex


In [85]:
path_plus_filename = None

In [63]:
filename = pset_df.loc[pset_ID, 'filename'][:-4]
title = (filename, 'pset ID: ' + str(pset_ID), 'Print to directories')
build_problem_df_tex(p_df, filename, title)

In [11]:
f = df2.path_plus_filename.loc[22]
path_unit, filename = f.rsplit('/', 1)
ff = path_unit + '/pdf/' + filename
ff

'/Users/chris/GitHub/course-files/Geom2021/02-Midpoint+distance/pdf/2-5DNQ_Segments+area.tex'

In [46]:
f[:-4]

'/Users/chris/GitHub/course-files/Geom2021/02-Midpoint+distance/2-5DNQ_Segments+area'

In [59]:
f = df2.filename.loc[22]
f

'2-5DNQ_Segments+area.tex'

In [60]:
os.system('pdflatex -output-directory=pdf ' + f)

0

In [15]:
import pylatex

In [22]:
min_latex = (r"\documentclass{article}"
             r"\begin{document}"
             r"Hello, world!"
             r"\end{document}")

#from latex import build_pdf

# this builds a pdf-file inside a temporary directory
#pdf = pylatex.build_pdf(min_latex)

# look at the first few bytes of the header
#print(pdf)

In [37]:
my_doc = pylatex.Document('2-5DNQ_Segments+area')

In [38]:
print(my_doc.dumps())

\documentclass{article}%
\usepackage[T1]{fontenc}%
\usepackage[utf8]{inputenc}%
\usepackage{lmodern}%
\usepackage{textcomp}%
\usepackage{lastpage}%
%
%
%
\begin{document}%
\normalsize%
\end{document}


In [39]:
my_pdf = pylatex.Document.generate_pdf('2-5DNQ_Segments+area')

AttributeError: 'str' object has no attribute '_select_filepath'

In [21]:
help(pylatex.document)

Help on module pylatex.document in pylatex:

NAME
    pylatex.document - This module implements the class that deals with the full document.

DESCRIPTION
    ..  :copyright: (c) 2014 by Jelte Fennema.
        :license: MIT, see License for more details.

CLASSES
    pylatex.base_classes.containers.Environment(pylatex.base_classes.containers.Container)
        Document
    
    class Document(pylatex.base_classes.containers.Environment)
     |  Document(default_filepath='default_filepath', *, documentclass='article', document_options=None, fontenc='T1', inputenc='utf8', font_size='normalsize', lmodern=True, textcomp=True, microtype=None, page_numbers=True, indent=None, geometry_options=None, data=None)
     |  
     |  A class that contains a full LaTeX document.
     |  
     |  If needed, you can append stuff to the preamble or the packages.
     |  For instance, if you need to use ``\maketitle`` you can add the title,
     |  author and date commands to the preamble to make it work.


In [73]:
os.system('pdflatex ' + f)

0

In [7]:
out_file_df = build_pset_df_tex(pset_df.loc[[170, 71, 75]], problem_df)
out_file_df

,filename,unit
pset_ID,,
170,10-4bDN_Analytics-review.tex,10-Trig+similarity+analytics
71,5-9bDN_Multi-step.tex,05-Transformations
75,6-11DN_Transformations.tex,06-Analytic-geometry


In [8]:
for file in out_file_df.filename:
    print('pdflatex ' + file)
    os.system('pdflatex ../out/' + file)

pdflatex 10-4bDN_Analytics-review.tex
pdflatex 5-9bDN_Multi-step.tex
pdflatex 6-11DN_Transformations.tex


In [32]:
problem_df[problem_df.problem_set_ID.apply(lambda x: x in [153, 155])].sort_index()

,problem_set_ID,question,filename,q_len
problem_ID,,,,
2184,153,"A transformation is applied to a triangle, $\t...",9-3bDN_Composition.tex,1104
2185,153,Determine and state the transformation mapping...,9-3bDN_Composition.tex,674
2186,153,First reflect the trapezoid $BECA$ across the ...,9-3bDN_Composition.tex,577
2187,153,\newpage\n,9-3bDN_Composition.tex,9
2188,153,Two transformations have been applied to a tri...,9-3bDN_Composition.tex,948
2189,153,The quadrilateral $ROCK$ undergoes two transfo...,9-3bDN_Composition.tex,905
2197,155,Given $\triangle ABC$ and $\triangle EFG$ with...,9-4CW_Triangle-congruence-proof.tex,1297
2198,155,"Two parallel lines intersect a transversal, $...",9-4CW_Triangle-congruence-proof.tex,1858
2199,155,\newpage\n,9-4CW_Triangle-congruence-proof.tex,9


In [76]:
# DUPES
print('Num of newpage * : ',
    problem_df.groupby('question').file.count().sort_values().iloc[-4:].sum())
print('Num of >= triples: ',
    problem_df.groupby('question').file.count().sort_values().iloc[-67:-4].sum())
print('Num of doubles ~ : ',
    problem_df.groupby('question').file.count().sort_values().iloc[-275:-67].sum())

Num of newpage *:  496
Num of >= triples:  204
Num of doubles ~ :  416


In [135]:
problem_df.tail(5)

,problem_set_ID,question,file,q_len
problem_ID,,,,
2346,171,"What is the equation of a line through $(-7,10...",10-6DN_Tangent-situations.tex,104
2347,171,What is an equation of the perpendicular bisec...,10-6DN_Tangent-situations.tex,113
2348,171,\newpage\n,10-6DN_Tangent-situations.tex,9
2349,171,$\triangle ABC$ is shown with $m\angle C=90^\c...,10-6DN_Tangent-situations.tex,745
2350,171,The following diagram shows a pole BT 1.6 m ta...,10-6DN_Tangent-situations.tex,885


## Reinitialize dfs from course files

In [4]:
course_files, course_file_df = map_course_files(course_dir='/Users/chris/GitHub/course-files/Geom')
print('dict len, type: ', len(course_files), type(course_files), '\n df len: ', len(course_file_df), '\n')
course_file_df.unit.unique()

dict len, type:  13 <class 'dict'> 
 df len:  208 



array(['01-Intro', '02-Midpoint+distance', '03-Volume+angle-bisectors',
       '04-Transversals', '05-Transformations', '06-Analytic-geometry',
       '07-Similarity', '08-Area+volume', '09-Congruence-transformations',
       '10-Trig+similarity+analytics', '11-Algebra2-intro', 'Slides',
       'Trajectories'], dtype=object)

In [5]:
drop_units = ['11-Algebra2-intro', 'Slides', 'Trajectories']
for unit in drop_units:
    course_file_df.drop(course_file_df[course_file_df.unit == unit].index, inplace=True)
#course_file_df

In [4]:
# Initialize for testing
course_file_df = pd.read_csv('/Users/chris/GitHub/mathai/db/course_file_df.csv', index_col=0)
print('course_file_df: ', len(course_file_df), course_file_df.index.name, course_file_df.columns)
course_file_df.head(2)

course_file_df:  172 None Index(['unit', 'file_count', 'filename'], dtype='object')


,unit,file_count,filename
0,01-Intro,16,1-1CW_Measurement.tex
1,01-Intro,16,1-1DNQ_Distance+slope.tex


In [5]:
pset_df, problem_df = parse_course_files(course_file_df, course_dir='/Users/chris/GitHub/course-files/Geom')
pset_df.head(2)

,unit,filename,problem_count,problem_IDs,problems_list,problem_kind,head,body
problem_set_ID,,,,,,,,
0,01-Intro,1-1CW_Measurement.tex,8,"[0, 1, 2, 3, 4, 5, 6, 7]",[Given the line segment $\overline{PQ}$ shown ...,"[question, newpage, question, newpage, questio...","[\begin{document}\n, \subsubsection*{1-1 Class...","[\n, \item Given the line segment $\overline..."
1,01-Intro,1-1DNQ_Distance+slope.tex,4,"[8, 9, 10, 11]",[Write down the slope and $y$-intercept of eac...,"[question, question, newpage, question]","[\begin{document}\n, \subsubsection*{0.1 Quiz:...","[\n, \item Write down the slope and $y$-inte..."


In [6]:
all(pset_df[['unit', 'filename', 'problems_list', 'problem_kind', 'head', 'body']] == pset_df2[['unit', 'filename', 'problems_list', 'problem_kind', 'head', 'body']])

True

In [10]:
print('pset_df: ', len(pset_df), pset_df.index.name, pset_df.columns)
print('\nproblem_df: ', len(problem_df), problem_df.index.name, problem_df.columns)

pset_df:  172 problem_set_ID Index(['unit', 'filename', 'problem_count', 'problem_IDs', 'problems_list',
       'problem_kind', 'head', 'body'],
      dtype='object')

problem_df:  2443 problem_ID Index(['problem_set_ID', 'question', 'kind', 'filename', 'q_len'], dtype='object')


In [11]:
print('pset_df2: ', len(pset_df2), pset_df2.index.name, pset_df2.columns)
print('\nproblem_df2: ', len(problem_df2), problem_df2.index.name, problem_df2.columns)

pset_df2:  172 problem_set_ID Index(['unit', 'filename', 'problem_count', 'problems_list', 'problem_kind',
       'head', 'body', 'problem_IDs'],
      dtype='object')

problem_df2:  2443 problem_ID Index(['problem_set_ID', 'question', 'filename', 'q_len'], dtype='object')


In [7]:
all(pset_df[['unit', 'filename', 'problems_list', 'problem_kind', 'head', 'body', 'problem_IDs']] == pset_df2[['unit', 'filename', 'problems_list', 'problem_kind', 'head', 'body', 'problem_IDs']])

True

In [8]:
problem_df.sort_values('q_len', inplace=True)

In [9]:
all(problem_df[['problem_set_ID', 'question', 'filename', 'q_len']] == problem_df2[['problem_set_ID', 'question', 'filename', 'q_len']])

True

## Parse small subsets of course

In [74]:
course_file_df = pd.DataFrame({"unit":['01-Intro'], "file_count":[6], "filename":['1-3DN_Function-intercepts.tex']})
course_file_df

,unit,file_count,filename
0,01-Intro,6,1-3DN_Function-intercepts.tex


In [75]:
#pset_df[pset_df.file.str.startswith('1-3DN')]
pset_1_3DN_df = parse_course_files(course_file_df, course_dir='/Users/chris/GitHub/course-files/Geom')

In [78]:
pset_1_3DN_df[['path', 'unit', 'file']] = pset_1_3DN_df.filename.str.rsplit('/', n=2, expand=True)
new_cols = ['unit', 'file', 'problem_count', 'head', 'body', 'problems_list', 'path', 'filename']
pset_df = pset_df.reindex(columns=new_cols)

## Review problems

In [34]:
pset_df.unit.unique()

array(['01-Intro', '02-Midpoint+distance', '03-Volume+angle-bisectors',
       '04-Transversals', '05-Transformations', '06-Analytic-geometry',
       '07-Similarity', '08-Area+volume', '09-Congruence-transformations',
       '10-Trig+similarity+analytics'], dtype=object)

In [39]:
pset_df.groupby('unit').count()

,filename,problem_count,problems_list,problem_kind,head,body,problem_IDs
unit,,,,,,,
01-Intro,16,16,16,16,16,16,16
02-Midpoint+distance,18,18,18,18,18,18,18
03-Volume+angle-bisectors,12,12,12,12,12,12,12
04-Transversals,8,8,8,8,8,8,8
05-Transformations,18,18,18,18,18,18,18
06-Analytic-geometry,25,25,25,25,25,25,25
07-Similarity,27,27,27,27,27,27,27
08-Area+volume,23,23,23,23,23,23,23
09-Congruence-transformations,18,18,18,18,18,18,18


In [33]:
pset_df.head(20)

,unit,filename,problem_count,problems_list,problem_kind,head,body,problem_IDs
problem_set_ID,,,,,,,,
0,01-Intro,1-1CW_Measurement.tex,8,[Given the line segment $\overline{PQ}$ shown ...,"[question, newpage, question, newpage, questio...","[\begin{document}\n, \subsubsection*{1-1 Class...","[\n, \item Given the line segment $\overline...","[3, 5, 1, 6, 2, 7, 0, 4]"
1,01-Intro,1-1DNQ_Distance+slope.tex,4,[Write down the slope and $y$-intercept of eac...,"[question, question, newpage, question]","[\begin{document}\n, \subsubsection*{0.1 Quiz:...","[\n, \item Write down the slope and $y$-inte...","[10, 9, 8, 11]"
2,01-Intro,1-1DN_Algebra.tex,7,"[In the following two problems, solve for the ...","[question, question, question, question, newpa...","[\begin{document}\n, \subsubsection*{1-1 Do No...","[\n, \n, \item In the following two problems...","[16, 13, 15, 14, 17, 12, 18]"
3,01-Intro,1-1HW_Algebra.tex,8,"[In the following two problems, solve for the ...","[question, question, question, question, quest...","[\begin{document}\n, \subsubsection*{1-1 Homew...","[\n, \n, \item In the following two problems...","[24, 20, 22, 21, 19, 23, 25, 26]"
4,01-Intro,1-3DN_Function-intercepts.tex,6,"[I have a compass, ruler, protractor, notebook...","[question, question, question, newpage, questi...","[\begin{document}\n, \subsubsection*{1-3 Do No...","[ \item I have a compass, ruler, protractor, ...","[30, 27, 28, 31, 29, 32]"
5,01-Intro,1-3HW_Equilateral-triangle.tex,11,[Construct an equilateral triangle with $\over...,"[question, question, newpage, question, questi...","[\begin{document}\n, \subsubsection*{Homework ...","[\n, \item Construct an equilateral triang...","[35, 41, 40, 39, 36, 42, 38, 37, 33, 34, 43]"
6,01-Intro,1-4HW-Intro.tex,115,[The points where a line segment begins and en...,"[question, question, question, question, quest...","[\begin{document}\n, \subsubsection*{Quiz Corr...",[ \item The points where a line segment begin...,"[118, 100, 106, 109, 155, 123, 145, 127, 130, ..."
7,01-Intro,1-4HW_Segment-addition.tex,11,"[I have a compass, ruler, protractor, notebook...","[question, question, question, question, quest...","[\begin{document}\n, \subsubsection*{Homework ...","[ \item I have a compass, ruler, protractor...","[166, 165, 161, 159, 167, 169, 164, 163, 168, ..."
8,01-Intro,1-5CW_Angle-terminology.tex,8,"[I have a compass, ruler, protractor, notebook...","[question, question, question, question, quest...","[\begin{document}\n, \subsubsection*{Classwork...","[ \item I have a compass, ruler, protractor...","[175, 173, 170, 172, 171, 174, 177, 176]"


In [18]:
ps = range(1160, 1166)
problem_df.loc[ps]

,problem_set_ID,question,filename,q_len
problem_ID,,,,
1160,79,In this problem use the following theorem (cop...,6-12Exam-Analytic-geometry.tex,1026
1161,79,\newpage\n,6-12Exam-Analytic-geometry.tex,9
1162,79,\subsubsection*{Using the distance formula t...,6-12Exam-Analytic-geometry.tex,71
1163,79,In this problem use the following theorem (cop...,6-12Exam-Analytic-geometry.tex,1027
1164,79,\newpage\n,6-12Exam-Analytic-geometry.tex,13
1165,79,Given $\overleftrightarrow{AB}$ as shown on th...,6-12Exam-Analytic-geometry.tex,541


In [10]:
pset_df.loc[3,'problems_list']

['In the following two problems, solve for the value of $x$.\n  \\begin{multicols}{2}\n    \\begin{enumerate}\n      \\item   $x-5=12$ \\vspace{6cm}\n      \\item   $13-x=-3$ \\vspace{6cm}\n    \\end{enumerate}\n  \\end{multicols}\n    \\vspace{3cm}\n\n',
 'Given $g(x)=x^2-5x+15$. Simplify $g(0)$. \\vspace{3cm}\n',
 'Given $f(x)=3x-2$. Solve for $x$ such that for $f(x)=13$. \\vspace{4cm}\n',
 'Given $h(x)=x^2-x-12$. Solve $h(x)=0$. \\vspace{5cm}\n  \n\n',
 'Simplify each expression. (Leave it in radical form if necessary, not a decimal.)\n    \\begin{enumerate}\n      \\begin{multicols}{2}\n      \\item   $\\sqrt{25}$ \\vspace{1.5cm}\n      \\item   $\\sqrt{24}$ \\vspace{1.5cm}\n      \\end{multicols}\n    \\end{enumerate}\n    \\vspace{0.5cm}\n\n',
 '\\newpage\n\n',
 "The line $l$ has the equation $y=\\frac{3}{2}x-1$. \n\\begin{enumerate}\n  \\item Write down it's slope. $m=$\n  \\vspace{0.5cm}\n  \\item Write down it's $y$-intercept. $b=$\n  \\vspace{0.5cm}\n  \\item Is the point $(4

In [11]:
problem_df[(problem_df.question.str.contains('multicol'))].head()

,problem_set_ID,question,file,q_len
problem_ID,,,,
367,30,Solve for $x$ by factoring\n \begin{multico...,2-8DN_Constructions.tex,171
400,32,\emph{Factor and solve for $x$.}\n \begin{enu...,2-9Exam_area-distance.tex,172
2133,146,Simplify each radical.\n \begin{multicols}{2}...,8-9DN_Circle-equation.tex,183
2132,146,Expand each binomial-squared expression to the...,8-9DN_Circle-equation.tex,214
1209,81,"In the following two problems, solve for the v...",6-1DN_graphing.tex,226


In [14]:
problem_df[problem_df.question.str.contains(r'\\newpage')].groupby('q_len').count()

,problem_set_ID,question,file
q_len,,,
9,431,431,431
10,22,22,22
11,40,40,40
12,2,2,2
13,13,13,13


In [23]:
# Example queries
problem_df[problem_df.question.str.contains(r'\\newpage')].question.loc[6]
problem_df[problem_df.file.str.startswith('1-1')].sort_index()

KeyError: 6

In [22]:
problem_df.loc[2335]

problem_set_ID                                                  163
question          Given $\triangle ABC$, $D$ is the midpoint of ...
filename                                9-9Exam-transformations.tex
q_len                                                           548
Name: 2335, dtype: object

,problem_set_ID,question,file,q_len
problem_ID,,,,
0,0,Given the line segment $\overline{PQ}$ shown b...,1-1CW_Measurement.tex,732
1,0,\newpage\n,1-1CW_Measurement.tex,11
2,0,Given an angle with vertex $A$. Answer the que...,1-1CW_Measurement.tex,675
3,0,\newpage\n,1-1CW_Measurement.tex,9
4,0,Given the rectangle $ABCD$ shown below. Answer...,1-1CW_Measurement.tex,1164
5,0,\newpage\n,1-1CW_Measurement.tex,9
6,0,"The points shown are in a straight line, $\ove...",1-1CW_Measurement.tex,533
7,0,Given $\overleftrightarrow{QS}$ as shown on th...,1-1CW_Measurement.tex,680
8,1,Write down the slope and $y$-intercept of each...,1-1DNQ_Distance+slope.tex,433


In [11]:
# Look for string in pset body
pset_df[pset_df.body.apply(lambda x: 'subsect' in str(x))] #r'[\\item'

,unit,file,problem_count,head,body,problems_list,path,filename,problem_IDs
problem_set_ID,,,,,,,,,
6,01-Intro,1-4HW-Intro.tex,104,"[\begin{document}\n, \subsubsection*{Quiz Corr...",[ \item The points where a line segment beg...,[The points where a line segment begins and en...,/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/01-Intro...,"[44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 5..."
26,02-Midpoint+distance,2-7CW_Modeling.tex,8,"[\begin{document}\n, \subsubsection*{2.7 Cla...",[ \subsubsection*{Do Not Solve! Make a draw...,[The point $Q$ is the midpoint of $\overline{P...,/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/02-Midpo...,"[318, 319, 320, 321, 322, 323, 324, 325]"
30,02-Midpoint+distance,2-8DN_Constructions.tex,4,"[\begin{document}\n, \subsubsection*{2.8 Do ...",[ \item Complete the construction of an equ...,[Complete the construction of an equilateral t...,/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/02-Midpo...,"[351, 352, 353, 354]"
32,02-Midpoint+distance,2-9Exam_area-distance.tex,36,"[\begin{document}\n, \subsubsection*{2.9 Exam:...","[\n, \item Given the rectangle $ABCD$ shown be...","[Given the rectangle $ABCD$ shown below, with ...",/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/02-Midpo...,"[361, 362, 363, 364, 365, 366, 367, 368, 369, ..."
36,03-Volume+angle-bisectors,3-2CW_Angle-modeling.tex,7,"[\begin{document}\n, \subsubsection*{3.2 Cla...","[ \subsubsection*{Do Not Solve! \\\n, M...",[Two lines intersect making four angles: $\ang...,/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/03-Volum...,"[418, 419, 420, 421, 422, 423, 424]"
...,...,...,...,...,...,...,...,...,...
144,08-Area+volume,8-8Exam_Area+volume.tex,33,"[\begin{document}\n, \subsubsection*{8.8 Exam:...","[\n, \item Use the formulas for the area and...",[Use the formulas for the area and circumferen...,/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/08-Area+...,"[1995, 1996, 1997, 1998, 1999, 2000, 2001, 200..."
145,08-Area+volume,8-9CW_Circle-equation.tex,15,"[\begin{document}\n, \subsubsection*{8.9 Class...","[\n, \item What is the equation of a circle ...",[What is the equation of a circle with center ...,/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/08-Area+...,"[2028, 2029, 2030, 2031, 2032, 2033, 2034, 203..."
146,08-Area+volume,8-9DN_Circle-equation.tex,5,"[\begin{document}\n, \subsubsection*{8.9 Do No...","[\n, \item A circle centered at the origin i...",[A circle centered at the origin includes the ...,/Users/chris/GitHub/course-files/Geom,/Users/chris/GitHub/course-files/Geom/08-Area+...,"[2043, 2044, 2045, 2046, 2047]"


In [13]:
# CHECK Problem with itemize instead of enumerate
problem_df[problem_df.problem_set_ID == 30].sort_index()

,problem_set_ID,question,file,q_len
problem_ID,,,,
351,30,Complete the construction of an equilateral tr...,2-8DN_Constructions.tex,684
352,30,\newpage\n,2-8DN_Constructions.tex,11
353,30,$x^2+6x+5=0$\n,2-8DN_Constructions.tex,13
354,30,$x^2+5x=14$\n \end{multicols}\n\n\n,2-8DN_Constructions.tex,34


In [34]:
for q in problem_df[problem_df.problem_set_ID == 30].sort_index().question:
    print(q)

Complete the construction of an equilateral triangle with one side as $\overline{AB}$. \vspace{3cm}
    \begin{center}
    \begin{tikzpicture}
      \draw [-, thick] (0,0)--(0,6);
      \draw [fill] (0,0) circle [radius=0.05] node[left]{$A$};
      \draw [fill] (0,6) circle [radius=0.05] node[left]{$B$};
    \end{tikzpicture}
    \end{center} \vspace{3cm}
    \begin{enumerate}
      \item Identify two circles in the construction. For each, name the center of the circle and the radius.  \vspace{3cm}
      \item Assuming that the third vertex of the triangle is point $C$, explain why the distance from $A$ to $C$ is the same as the distance from $A$ to $B$.
    \end{enumerate}


  \newpage

$x^2+6x+5=0$

$x^2+5x=14$
    \end{multicols}





In [12]:
pset_df.loc[30]

unit                                          02-Midpoint+distance
file                                       2-8DN_Constructions.tex
problem_count                                                    5
head             [\begin{document}\n,   \subsubsection*{2.8 Do ...
body             [\item Complete the construction of an equilat...
problems_list    [Complete the construction of an equilateral t...
path                         /Users/chris/GitHub/course-files/Geom
filename         /Users/chris/GitHub/course-files/Geom/02-Midpo...
problem_IDs                              [364, 366, 367, 365, 363]
Name: 30, dtype: object

In [23]:
pset_df[pset_df.problems_list.apply(len) != pset_df.problem_kind.apply(len)]

,filename,head,body,problems_list,problem_kind,problem_count,path,unit,file
problem_set_ID,,,,,,,,,


In [16]:
problem_df[problem_df.q_len > 16].head(20)

,problem_set_ID,question,kind,filename,q_len
problem_ID,,,,,
1647,116,\subsubsection*{Spicy}\n,section,7-7Exam-Similarity.tex,23
91,6,Find the value of $|2-7|$.\n\n,question,1-4HW-Intro.tex,28
61,6,Find the value of $|8.5-3|$.\n\n,question,1-4HW-Intro.tex,30
816,57,\subsubsection*{Early finishers}\n,section,5-11bExam_Transformations.tex,33
777,56,\subsubsection*{Early finishers}\n,section,5-11Exam_Transformations.tex,33
1011,73,\subsubsection*{Early finishers}\n,section,6-10bCW_Dilation.tex,33
1303,88,\subsubsection*{Early finishers}\n,section,6-7EN_Midpoint-graphs.tex,33
1062,76,\subsubsection*{Early finishers}\n,section,6-11HW_pretest-angles.tex,33
1400,95,\subsubsection*{Early finishers}\n,section,6-9HW_pretest-graphing.tex,33


In [22]:
problem_df.groupby('kind').count()

,problem_set_ID,question,filename,q_len
kind,,,,
newpage,509,509,509,509
question,1813,1813,1813,1813
section,121,121,121,121


In [21]:
problem_df

,problem_set_ID,question,kind,filename,q_len
problem_ID,,,,,
0,0,Given the line segment $\overline{PQ}$ shown b...,question,1-1CW_Measurement.tex,732
1,0,\newpage\n\n,newpage,1-1CW_Measurement.tex,10
2,0,Given an angle with vertex $A$. Answer the que...,question,1-1CW_Measurement.tex,676
3,0,\newpage\n,newpage,1-1CW_Measurement.tex,9
4,0,Given the rectangle $ABCD$ shown below. Answer...,question,1-1CW_Measurement.tex,1165
...,...,...,...,...,...
2438,171,"What is the equation of a line through $(-7,10...",question,10-6DN_Tangent-situations.tex,104
2439,171,What is an equation of the perpendicular bisec...,question,10-6DN_Tangent-situations.tex,113
2440,171,\newpage\n\n,newpage,10-6DN_Tangent-situations.tex,10


In [27]:
pset_df.sort_values('problem_count')[['filename', 'problem_count']].tail(30)

,filename,problem_count
problem_set_ID,,
161,9-8DN-Similarity-review.tex,20
132,8-2bDN_Estimation.tex,22
115,7-6bCW_Tangent+mixed.tex,23
143,8-7bPreExam_Area+volume.tex,23
13,1-7Exam-corrections.tex,23
71,5-9bDN_Multi-step.tex,24
12,1-7Exam-Intro.tex,25
160,9-7CW-Similarity-review.tex,25
117,7-7bExam-Similarity.tex,26
